In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
data = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic", "time", "imglink"])

In [3]:

#url dùng để thu thập dữ liệu từ báo Lao Động
u1 = "https://laodong.vn/xa-hoi?page="
u2 = "https://laodong.vn/the-thao?page="
u3 = "https://laodong.vn/van-hoa?page="
u4 = "https://laodong.vn/kinh-doanh?page="
u5 = "https://laodong.vn/the-gioi?page="
u6 = "https://laodong.vn/thoi-su?page="
u7 = "https://laodong.vn/cong-doan?page="
u8 = "https://laodong.vn/giai-tri?page="

In [6]:
def laodong_topic(web_link):
    if "xa-hoi" in web_link:
        return "Xã hội"
    elif "the-thao" in web_link:
        return "Thể thao"
    elif "van-hoa" in web_link:
        return "Văn hóa"
    elif "giai-tri" in web_link:
        return "Giải trí"
    elif "kinh-doanh" in web_link:
        return "Kinh doanh"
    elif "the-gioi" in web_link:
        return "Thế giới"
    elif "thoi-su" in web_link:
        return "Thời sự"
    elif "cong-doan" in web_link:
        return "Công đoàn"
    else:
        return "Unknown category"

In [5]:
from urllib.parse import urlparse, parse_qs

In [7]:
#lấy tiêu đề
def find_all_articles_at_laodong(max_sl, web_link, data):
  for i in range(1, max_sl + 1):
    url = web_link + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    all_articles = soup.find_all('article', class_='v4 p2c m002')
    for article in all_articles:
      title = article.select_one('a.link-title h2.title').get_text(strip=True)
      href = article.select_one('a.link-title')['href']
      if article.select_one('div.chapeau'):
        abstract = article.select_one('div.chapeau').get_text(strip=True)
      else:
        response2 = requests.get(href)
        soup2 = BeautifulSoup(response2.content, "html.parser")
        abstract = soup2.find('div', class_='chappeau').get_text(strip=True)
      response2 = requests.get(href)
      soup2 = BeautifulSoup(response2.content, "html.parser")
      time = soup2.find('span', class_ = 'time')
      if time:
        time = time.get_text(strip=True).split(',')[1].strip().split(' ')
        time = time[0] + " " + time[1]
      image_link = article.select_one('img')['src']
      parsed_url = urlparse(image_link)
      image_link = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
      data = data.append({"title" : title, "abstract": abstract, "source": "Báo Lao Động", "link" : href, "topic": laodong_topic(web_link), "time" : time, "imglink" : image_link}, ignore_index=True)
  return data

In [8]:

#craft time
data = find_all_articles_at_laodong(20, u1, data)
data = find_all_articles_at_laodong(20, u2, data)
data = find_all_articles_at_laodong(20, u3, data)
data = find_all_articles_at_laodong(20, u4, data)
data = find_all_articles_at_laodong(20, u5, data)
data = find_all_articles_at_laodong(20, u6, data)
data = find_all_articles_at_laodong(20, u7, data)
data = find_all_articles_at_laodong(20, u8, data)
data = data.drop_duplicates()
data

,title,abstract,source,link,topic,time,imglink


In [9]:
data.to_csv('laodong.csv', index = False, encoding = 'utf-8-sig')